In [1]:
from gensim.models import Word2Vec, KeyedVectors, TfidfModel
from gensim.parsing.preprocessing import STOPWORDS
from scipy.sparse.linalg import svds
from scipy.spatial.distance import cosine
import numpy as np
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import pandas as pd

In [2]:
df_train = pd.read_csv("/home/users/kostadin.mishev/phd/dataset/train/train.tsv",names=["id","sentiment","a","text"],sep='\t')
df_dev = pd.read_csv("/home/users/kostadin.mishev/phd/dataset/dev/dev.tsv",names=["id","sentiment","a","text"],sep='\t')
train_text = df_train['text'].values
test_text = df_dev['text'].values

In [3]:
import keras
Y_train = df_train['sentiment'].values
Y_dev = df_dev['sentiment'].values

Y_train = keras.utils.to_categorical(Y_train,2)
Y_dev = keras.utils.to_categorical(Y_dev,2)

Y_dev_labels = np.argmax(Y_dev,axis=1)

Using TensorFlow backend.
/home/users/kostadin.mishev/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/users/kostadin.mishev/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/users/kostadin.mishev/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/users/kostadin

In [4]:
import gensim.downloader as api
word2vec_model = api.load("glove-wiki-gigaword-200")

KeyboardInterrupt: 

In [5]:
model = Doc2Vec.load("/home/users/kostadin.mishev/phd/finsent/doc2vec/enwiki_dbow/doc2vec.bin") 

In [6]:
X_train = []
for text in train_text:
    X_train.append(model.infer_vector(text.split()))
X_dev = []
for text in test_text:
    X_dev.append(model.infer_vector(text.split()))

In [7]:
X_train= np.asarray(X_train)
X_dev = np.asarray(X_dev)

In [8]:
Y_train_labels = np.argmax(Y_train,axis=1)
Y_dev_labels = np.argmax(Y_dev,axis=1)

In [9]:
X_dev.shape

(438, 300)

In [16]:
from sklearn.svm import LinearSVC
from sklearn.svm import SVC # "Support vector classifier"
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef

for c in [0.0025,0.025,0.1,0.25,1,10,50,100,150,200,1000,2000,5000]:
    print(c)
    model = SVC(kernel='linear', C=c, gamma=0.001)
    model.fit(X_train, Y_train_labels)
    Y_pred = model.predict(X_dev)
    tn, fp, fn, tp = confusion_matrix(Y_dev_labels,Y_pred).ravel() 
    mcc = matthews_corrcoef(Y_dev_labels, Y_pred)

    print(tp)
    print(tn)
    print(fp)
    print(fn)
    print(mcc)
    print()

0.0025
73
211
8
146
0.382240534018029

0.025
100
201
18
119
0.42198582191621686

0.1
137
185
34
82
0.48204051173447887

0.25
145
183
36
74
0.5053830229776322

1
149
183
36
70
0.5223147840931739

10
152
182
37
67
0.530111547606542

50
147
177
42
72
0.4840148912929297

100
145
176
43
74
0.47049091609473137

150
145
174
45
74
0.4606778794979673

200
147
172
47
72
0.4596256127589121

1000


KeyboardInterrupt: 

In [13]:
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef

#Train the XGboost Model for Classification
model1 = xgb.XGBClassifier(n_estimators=1000, max_depth=8, learning_rate=0.01)

xgb_model = model1.fit(X_train, Y_train_labels)

Y_pred = xgb_model.predict(X_dev)

tn, fp, fn, tp = confusion_matrix(Y_dev_labels,Y_pred).ravel() 
mcc = matthews_corrcoef(Y_dev_labels, Y_pred)

print(tp)
print(tn)
print(fp)
print(fn)
print(mcc)

163
163
56
56
0.4885844748858447


In [20]:
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
params = {"learning_rate"    : [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
 "max_depth"        : [ 3, 4, 5, 6, 8, 10, 12, 15],
 "min_child_weight" : [ 1, 3, 5, 7 ],
 "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
 "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] }

xgb_model= xgb.XGBClassifier()

clf = GridSearchCV(xgb_model, params)

In [21]:
clf.fit(X_train, Y_train_labels)

GridSearchCV(cv=None, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='binary:logistic',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale...t=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=None,
             param_grid={'colsample_bytree': [0.3, 0.4, 0.5, 0.7],
                         'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                         'learning_rate': [0.05, 0.1,

In [33]:
from tensorflow.keras import layers;
import tensorflow

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
np.random.seed(7)
from tensorflow.keras import layers;

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten

from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef


file_path = "./Doc2Vec_MLP.hdf5"
ckpt = ModelCheckpoint(file_path, monitor='val_loss', verbose=1,
                       save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=5)

vector=300

batch_size = 16
epochs = 100


def Doc2VecMLP(vector):
  model_use_sent = Sequential()
  #model_use_sent.add(Dense(512, activation='relu', input_shape=(vector,)))
  #model_use_sent.add(Dropout(0.5))
  model_use_sent.add(Dense(128, activation='relu', input_shape=(vector,)))
  model_use_sent.add(Dropout(0.5))
  model_use_sent.add(Dense(32, activation='relu'))
  model_use_sent.add(Dropout(0.5))
  model_use_sent.add(Dense(2, activation='softmax'))
  #model.summary()
  optimizer = tensorflow.keras.optimizers.Adam(lr=0.001)
  model_use_sent.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy']) # compile model with training parameters
  return model_use_sent;

def InferSentCNN(vector,num_filters):
  model = Sequential()
  model.add(Conv1D(num_filters, 20, activation='relu', padding='same',input_shape=(vector,1)) )
  model.add(MaxPooling1D(2))
  model.add(Conv1D(num_filters, 20, activation='relu', padding='same'))
  model.add(GlobalMaxPooling1D())
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dense(2, activation='sigmoid'))
  optimizer = tensorflow.keras.optimizers.Adam(lr=0.001)
  model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
  return model;


In [35]:
mcc=list()
tp=list()
tn=list()
fp=list()
fn=list();

for i in range(20):
  model_use_sent = Doc2VecMLP(vector)
  model_use_sent.summary()
  history = model_use_sent.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_dev,Y_dev), callbacks=[ckpt, early],verbose=0)
  prediction = model_use_sent.predict(X_dev)

  Y_pred = np.argmax(prediction,axis=1)
  tn_, fp_, fn_, tp_ = confusion_matrix(Y_dev_labels,Y_pred).ravel() 
  mcc_ = matthews_corrcoef(Y_dev_labels, Y_pred)
  tp.append(tp_)
  tn.append(tn_)
  fp.append(fp_)
  fn.append(fn_)
  mcc.append(mcc_)
  print("ITERATION:"+str(i))


  print(accuracy_score(Y_dev_labels, Y_pred))
  print(precision_score(Y_dev_labels, Y_pred))
  print(recall_score(Y_dev_labels, Y_pred))
  print(f1_score(Y_dev_labels, Y_pred))
  print(mcc_)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 128)               38528     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                4128      
_________________________________________________________________
dropout_3 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 66        
Total params: 42,722
Trainable params: 42,722
Non-trainable params: 0
_________________________________________________________________

Epoch 00001: val_loss improved from inf to 0.61426, saving model to ./Doc2Vec_MLP.hdf5

Epoch 00002: val_loss improved from 0.61426 to 0.


Epoch 00001: val_loss did not improve from 0.45501

Epoch 00002: val_loss did not improve from 0.45501

Epoch 00003: val_loss did not improve from 0.45501

Epoch 00004: val_loss did not improve from 0.45501

Epoch 00005: val_loss did not improve from 0.45501

Epoch 00006: val_loss did not improve from 0.45501

Epoch 00007: val_loss did not improve from 0.45501

Epoch 00008: val_loss did not improve from 0.45501

Epoch 00009: val_loss did not improve from 0.45501

Epoch 00010: val_loss did not improve from 0.45501

Epoch 00011: val_loss did not improve from 0.45501

Epoch 00012: val_loss did not improve from 0.45501

Epoch 00013: val_loss did not improve from 0.45501
ITERATION:4
0.7579908675799086
0.7729468599033816
0.730593607305936
0.7511737089201879
0.5167580857740227
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_19 (Dense)             (None, 128)               38528     
____________________


Epoch 00014: val_loss did not improve from 0.45501
ITERATION:8
0.7831050228310502
0.7952380952380952
0.7625570776255708
0.7785547785547785
0.5666887803439845
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 128)               38528     
_________________________________________________________________
dropout_20 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_32 (Dense)             (None, 32)                4128      
_________________________________________________________________
dropout_21 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_33 (Dense)             (None, 2)                 66        
Total params: 42,722
Trainable params: 42,722
Non-trainable params: 0
_____________________________________________


Epoch 00001: val_loss did not improve from 0.45476

Epoch 00002: val_loss did not improve from 0.45476

Epoch 00003: val_loss did not improve from 0.45476

Epoch 00004: val_loss did not improve from 0.45476

Epoch 00005: val_loss did not improve from 0.45476

Epoch 00006: val_loss did not improve from 0.45476

Epoch 00007: val_loss did not improve from 0.45476

Epoch 00008: val_loss did not improve from 0.45476

Epoch 00009: val_loss did not improve from 0.45476

Epoch 00010: val_loss did not improve from 0.45476

Epoch 00011: val_loss did not improve from 0.45476

Epoch 00012: val_loss did not improve from 0.45476

Epoch 00013: val_loss did not improve from 0.45476

Epoch 00014: val_loss did not improve from 0.45476

Epoch 00015: val_loss did not improve from 0.45476
ITERATION:13
0.7648401826484018
0.7685185185185185
0.7579908675799086
0.7632183908045977
0.5297300702061513
_________________________________________________________________
Layer (type)                 Output Shape     

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_58 (Dense)             (None, 128)               38528     
_________________________________________________________________
dropout_38 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_59 (Dense)             (None, 32)                4128      
_________________________________________________________________
dropout_39 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_60 (Dense)             (None, 2)                 66        
Total params: 42,722
Trainable params: 42,722
Non-trainable params: 0
_________________________________________________________________

Epoch 00001: val_loss did not improve from 0.45476

Epoch 00002: val_loss did not improve from 0.45476

Epoch 00003: val_loss did not imp

In [36]:
print(np.mean(tp))
print(np.mean(tn))
print(np.mean(fp))
print(np.mean(fn))
print(np.mean(mcc))

160.45
173.75
45.25
58.55
0.5286192415127409


In [ ]:
1